# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении мод
ели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального
заёмщика вернуть кредит банку

## Оглавление
1. [Просмотр данных](#start)
2. [Предобработка данных](#preprocessing)
      * [Обработка пропущенных значений](#null)
      * [Замена типа данных](#replace)
      * [Обработка дубликатов](#dublicates)
      * [Лемматизация](#lemma)
      * [Категоризация данных](#class)
3. [Анализ данных](#analysis)
      * [Выявление зависмости между наличием детей и возвратом кредита в срок](#children)
      * [Выявление зависмости между семейным положением и возвратом кредита в срок](#family)
      * [Выявление зависмости между уровнем дохода и возвратом кредита в срок](#income)
      * [Выявление зависмости как разные цели кредита влияют на его возврат в срок](#purpose)
4. [Общий вывод по проекту](#inference)

## 1. Просмотр данных. <a id="start"></a>

Прежде всего получим общую информацию о файле с данными.

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Из полученных данных видим, что кол-во записей в 2 колонках **days_employed** и **total_income** меньше, чем в остальных. Значит в этих колонках есть пропущенные данные. Общее кол-во записей в колонках должно быть 21525

In [2]:
from IPython.display import display
display(data.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


При просмотре первых 15 записей видим, что в таблице есть отрицательные значения, названия с верхним регистром и колонки с дробными цифрами. Nan так же подтверждает наличие пропусков. 

### Вывод

Необходима предварительная обработка данных до анализа.

## 2. Предобработка данных <a id="preprocessing"></a>

### Обработка пропущенных значений  <a id="null"></a>

Для начала найдем общее кол-во пропущенных значений

In [3]:
nan_number = data.isna().sum()
print(nan_number)

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Вычислим долю пропущенных значений от общего числа записей

In [4]:
print(2174/21525*100 )

10.099883855981417


Доля пропущенных данных в таблице составляет 10%. Это довольно много и может привести к искаженному анализу. Такое одинаковое кол-во пропущенных значений, и при этом всего в 2-х колонках, говорит о технической ошибке либо при вводе данных либо при выгрузке 

Пропущенные значения заполним медианой в зависимости от типа занятости, тк доходы и трудовой стаж студентов и работающих людей, например, могут значительно отличаться 

In [5]:
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))
display(data.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Некоторые данные в колонке трудовой стаж **days_employed** имеют отрицательные значения. Большое количество таких значений указывает на техническую ошибку сбора данных. Перед расчетом медианы необходимо привести числа к абсолютному значению.

In [6]:
data['days_employed'] = abs(data['days_employed'])
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('median'))
display(data.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Пропущенные значения заполнены медианными данными по типу занятости. 

Проверим что кол-во записей одинаково для всех колонок

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


21525 записей во всех колонках.

### Вывод

  Пропуски заполнены. 

### Замена типа данных  <a id="replace"></a>

Для анализа понадобятся данные о количестве детей. Необходимо проверить выборку данных по колонке **children**

In [8]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


В списке выбиваются числа -1 и 20. Количество детей не может быть отрицательным и 20 детей выглядит маловероятным. Учитывая не единичное количество строк с такими данными, маловероятно что это опечатка. К тому же нет явных ошибок по другим числам. Скорее всего, техническая ошибка выгрузки данных. Суммарно таких записей 123.  Это примерно 0,5% от общего кол-ва, поэтому можно заменить даннные. 

Ошибочные данные заменяются медианой. Перед расчетом медианы надо их удалить, иначе они исказят результат.

In [9]:
data.loc[data['children'] == -1,['children']] = 'nan'
data.loc[data['children'] == 20,['children']] = 'nan'
data_children_median = data['children'].median()
print(data_children_median)

0.0


По медиане получилось 0 детей. Сделаем замену в удаленных значениях

In [10]:
data.loc[data['children'] == 'nan',['children']] = data_children_median
data['children'] = data['children'].astype('int')
print(data['children'].value_counts())

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


Теперь в списке от 0 до 5 детей, что выглядит правдободобно учитавая, что кол-во заемщиков уменьшается с ростом детей.

Необходимо проверить данные, которые не могут быть равны нулю. В данной таблице это возраст, колонка  **dob_years**

In [11]:
display(data.sort_values(by='dob_years',ascending=True).head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6859,1,260.992722,0,неоконченное высшее,2,гражданский брак,1,F,сотрудник,0,128246.646591,покупка недвижимости
16042,2,176.404487,0,среднее,1,женат / замужем,0,F,сотрудник,0,137563.630693,жилье
7034,0,366067.781030,0,высшее,0,Не женат / не замужем,4,F,пенсионер,0,263121.074528,образование
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости
4147,0,3549.117333,0,среднее,1,в разводе,3,M,сотрудник,0,108130.933212,покупка жилой недвижимости


Как видно, нули в выборке присутвуют. Возможно, возраст забыли указать при заполнении данных. Нули заменяются медианой из выборки. Так же проверим, что нет заемщиков младше 18 лет, тк это тоже может быть ошибкой

In [12]:
count = 0
for row in data['dob_years']:
    if row >0 and row <18 :
        count += 1
print(count)     

0


 Заемщиков младше 18 нет, поэтому можно рассчитать медиану

In [13]:
data_age_median = data['dob_years'].median()
data_age_median 
print(data_age_median)

42.0


Заменим строки с нулевым возрастом на значение медианы = 42 года

In [14]:
data.loc[data['dob_years'] == 0,['dob_years']] = data_age_median

Проверим что больше не осталось значений = 0

In [15]:
print(data[data['dob_years']==0])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


Данные по возврасту исправлены.

Так же при просмотре данных были выявлены колонки с дробными числами

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


В 3-х столбцах days_employed, dob_years и total_income  тип данных - дробные. С такими данными неудобно работать. Приведем их к целочисленному типу 

In [17]:
data['dob_years'] = data['dob_years'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Тип данных исправлен на целые числа

### Вывод

Все столбцы с числовыми значениямим приведены к типу целых чисел и отсутвуют нулевые значкения, которые могут повлиять на анализ данных.

### Обработка дубликатов <a id="dublicates"></a>

При предварительном просмотре данных таблицы были обнаружены названия с заглавными буквами, которые могут быть дубликатами названий, написанных строчными буквами. 
Дубликаты с заглавными буквами появились в результате разного формата ввода данных у тех кто собирает их в самом начале у заемщиков.

Проверим стобцы с типом данных object: **purpose, family_status, income_type, education, gender**.

Для этого сгруппируем все возможные значения в таблице

In [18]:
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Все слова написаны строчными бкувами. Не нужно исправлять

In [19]:
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


Есть названия с верхним регистром. Нужно исправить.

In [20]:
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


Все слова написаны строчными бкувами. Не нужно исправлять

In [21]:
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


Есть названия с верхним регистром и из-за этого появляется много дубликатов. Нужно исправить.

В колонках **education** и **family_status** приведем все к нижнему регистру. 

In [22]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

Проверим исправления

In [23]:
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [24]:
print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Все слова написаны в одном формате, кол-во наименований уменьшилось

Проверим последний столбец пол **gender**

In [25]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Появляется 3-й пол XNA. Вероятно, опечатка, тк в единичном кол-ве. Тк значений F больше, то меняем XNA на F. 

In [26]:
data.loc[data['gender'] == 'XNA',['gender']] = 'F'
print(data['gender'].value_counts())

F    14237
M     7288
Name: gender, dtype: int64


Проверим наличие грубых дубликатов - строки с идентичными значениями.

In [27]:
print(data.duplicated().sum())

72


Проверка на дубликаты показывает наличие строк с идентичными данным. Так как это грубые дубликаты, те повторяются все значения в стоках, то вряд ли это разные люди. Это может быть техническая ошибка выгрузки данных.

Удалим дубликаты.

In [28]:
data = data.drop_duplicates().reset_index(drop=True)
print(data.duplicated().sum())

0


### Вывод

В таблице удалены дубликаты

### Лемматизация <a id="lemma"></a>

Для анализва данных понадобится колонка с целями кредита - **purpose** Но в ней очень много одинаковых значений, написанных разными словами. Их необходимо объединить в категории. Для выделения категорий по целям кридита  определим ключевые слова и частоту их употребления.

In [29]:
from pymystem3 import Mystem
m = Mystem() 
lemmas = []
for row in data['purpose']:
    lemmas.extend(m.lemmatize(row))
  
from collections import Counter
print(Counter(lemmas))

Counter({' ': 33568, '\n': 21453, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2323, 'свой': 2230, 'на': 2221, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 767, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})



В списке слов можно выделить 2 главные группы: объект и действия над объектом. Есть еще определения (подержаный, дополнительный и тп),но они только характеризуют объекты. Действие это: покупка, получение, приобретение и тд. Действия совершают над объектами. 

Для выделения групп нужны объекты. Самые частоупотребляемые: недвижимость, автомобиль, обозование, жилье(приравнивается к недвижимости), свадьба. Есть еще 'операция'. Это может быть как действие так и объект.  

Проверим с какими словами сочетается 'операция'

In [30]:
data_selected = data['purpose'].loc[data['purpose'].str.contains('операц')].value_counts()
print(data_selected)

операции с недвижимостью                 675
операции с жильем                        652
операции с коммерческой недвижимостью    650
операции со своей недвижимостью          627
Name: purpose, dtype: int64


Как видно, 'операция' здесь действие

### Вывод

Лемматизация позволила выделить 4 слова **недвижимость, автомобиль, обозование, свадьба**, по которым можно сгруппировать цели кредита.

### Категоризация данных <a id="class"></a>

Объединим в категории данные, которые будут ипользоваться для анализа. 
Основый признак, по которому проводится анализ - наличие задолженности по кредиту
Поэтому сначала выделим категории по колонке наличие задолженности **debt**

In [31]:
def debt_group(debt):
    try:
        if debt == 0:
                return 'задолженности нет'
        if debt == 1:
                return 'задолженность есть'
    except:
        print('ошибочное значение')
data['debt_group'] = data['debt'].apply(debt_group)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,debt_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,задолженности нет
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,задолженности нет
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,задолженности нет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,задолженности нет
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,задолженности нет


В код включен цикл для проверки на ошибку, если в выборке попадутся какие то еще значения, кроме 0 и 1

Теперь объединим в категории цели кредита по колонке **purpose**

In [32]:
def purpose_group(purpose):
    try:
        if 'недвижимост' in purpose:
            return 'недвижимость'
        if 'образован' in purpose:
            return 'образование'
        if 'автомоб' in purpose:
            return 'автомобиль'
        if 'свадьб' in purpose:
            return 'свадьба'
        if 'жиль'in purpose:
            return 'недвижимость'
    except:
        print('нет совпадений')   
        
data['purpose_group'] = data['purpose'].apply(purpose_group)
display(data.head())        

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,debt_group,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,задолженности нет,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,задолженности нет,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,задолженности нет,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,задолженности нет,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,задолженности нет,свадьба


В таблицу добавлены категории по целям кредита. С циклом проверки на ошибки, на случай если в ячейке не будет совпадений. 

Так как нужен будет сравнительный анализ по наличию детей. Выделим общие категории по наличию или отсутствию детей по колонке **children**

In [33]:
def children_group(children):
 
        if children == 0:
                return 'детей нет'
        if children > 0:
                return 'дети есть'
data['children_group'] = data['children'].apply(children_group)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,debt_group,purpose_group,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,задолженности нет,недвижимость,дети есть
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,задолженности нет,автомобиль,дети есть
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,задолженности нет,недвижимость,детей нет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,задолженности нет,образование,дети есть
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,задолженности нет,свадьба,детей нет


Цикл проверки на ошибки не нужен, тк эти данные были обработаны и исправлены раньше

Объединим в группы размер ежемесячного дохода по колонке **income**.
По данным Росстата средняя з\п в 2020 году составляла 40-50 тыс.
Можно оттолкнуться от 50 тыс для группировки данных:

    < 50K
    от 50К до 100К
    от 100К до 200К
    от 200К до 500К
    > 500K

In [34]:
def income_group(total_income):
 
        if total_income <= 50000:
                return 'доход < 50К'
        if total_income <= 100000:
                return 'доход от 50К до 100К'
        if total_income <= 200000:  
                return 'доход от 100К до 200К'
        if total_income <= 500000:  
                return 'доход от 200К до 500К'
        if total_income > 500001:  
                return 'доход > 500К '    
data['income_group'] = data['total_income'].apply(income_group)
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,debt_group,purpose_group,children_group,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,задолженности нет,недвижимость,дети есть,доход от 200К до 500К
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,задолженности нет,автомобиль,дети есть,доход от 100К до 200К
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,задолженности нет,недвижимость,детей нет,доход от 100К до 200К
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,задолженности нет,образование,дети есть,доход от 200К до 500К
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,задолженности нет,свадьба,детей нет,доход от 100К до 200К
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,задолженности нет,недвижимость,детей нет,доход от 200К до 500К
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,задолженности нет,недвижимость,детей нет,доход от 200К до 500К
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,задолженности нет,образование,детей нет,доход от 100К до 200К
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,задолженности нет,свадьба,дети есть,доход от 50К до 100К
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,задолженности нет,недвижимость,детей нет,доход от 100К до 200К


 Цикл проверки не испрользуем, тк фильтр предпогает проверку сумм от 0 до бесконечности

Проврим что в таблицу добавились все значения в новых колонках. 

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 16 columns):
children            21453 non-null int64
days_employed       21453 non-null int64
dob_years           21453 non-null int64
education           21453 non-null object
education_id        21453 non-null int64
family_status       21453 non-null object
family_status_id    21453 non-null int64
gender              21453 non-null object
income_type         21453 non-null object
debt                21453 non-null int64
total_income        21453 non-null int64
purpose             21453 non-null object
debt_group          21453 non-null object
purpose_group       21453 non-null object
children_group      21453 non-null object
income_group        21453 non-null object
dtypes: int64(7), object(9)
memory usage: 2.6+ MB


21453 записей во всех колонках. Пропусков нет.

### Вывод

Данные, по которым будет производится анализ, разбиты на категории.

## 3.   Анализ данных <a id="analysis"></a>

Для анализа сгруппируем данные в сводные таблицы. Будем анализировать кол-во заемщиков, имеющих задолженность по кредиту, по разным группам. И на основе полученных данных выявим от чего может зависеть задолженность.

### Выявление зависимости между наличием детей и возвратом кредита в срок <a id="children"></a>

In [36]:
data_pivot = data.pivot_table(index='children', columns='debt_group', values='debt', aggfunc='count')
display(data_pivot)

debt_group,задолженности нет,задолженность есть
children,,
0,13141.0,1072.0
1,4364.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


На первый взгляд кажется что те, у кого нет детей имеют больше задолженностей.
Но посмотрим на долю задолженностей из общего числа.

In [37]:
data_pivot['debt_part'] = data_pivot['задолженность есть'] / (data_pivot['задолженность есть'] + data_pivot['задолженности нет'])
display(data_pivot.sort_values(by='debt_part',ascending = False))

debt_group,задолженности нет,задолженность есть,debt_part
children,,,
4,37.0,4.0,0.097561
2,1858.0,194.0,0.094542
1,4364.0,444.0,0.092346
3,303.0,27.0,0.081818
0,13141.0,1072.0,0.075424
5,9.0,NaN,NaN


Самая большая доля задолженности у заемщиков с 4 детьми. Но какой то общей тенденции не видно. Проверим по общей группе наличия или отсутсвия детей.

In [38]:
data_pivot2 = data.pivot_table(index='children_group', columns='debt_group', values='debt', aggfunc='count')
data_pivot2['debt_part'] = data_pivot2['задолженность есть'] / (data_pivot2['задолженность есть'] + data_pivot2['задолженности нет'])
print(data_pivot2.sort_values(by='debt_part',ascending = False))

debt_group      задолженности нет  задолженность есть  debt_part
children_group                                                  
дети есть                    6571                 669   0.092403
детей нет                   13141                1072   0.075424


Самая большая доля задолженностей у заемщиков с детьми. Зависимость от кол-ва детей отсутствует. Заемщики с 1 ребенком чаще имеют просрочку чем заемщики с 3 детьми.

У семей с детьми увеличиваются расходы, в том числе и непредвиденные, и доход на 1 человека в семье уменьшается, если дети несовершеннолетние и не работают

### Вывод

Наличие детей потенциально влияет на способность вернуть кредит в срок. Хотя прямой зависимости от кол-ва детей нет. 

### Выявление зависимости между семейным положением и возвратом кредита в срок <a id="family"></a>

In [39]:
data_pivot_family = data.pivot_table(index='family_status', columns='debt_group', values='debt', aggfunc='count')


In [40]:
data_pivot_family['part_familiy_debt'] = data_pivot_family['задолженность есть'] / data_pivot_family['задолженности нет']
display(data_pivot_family.sort_values(by='part_familiy_debt', ascending = False))

debt_group,задолженности нет,задолженность есть,part_familiy_debt
family_status,,,
не женат / не замужем,2536,274,0.108044
гражданский брак,3762,388,0.103137
женат / замужем,11408,931,0.081609
в разводе,1110,85,0.076577
вдовец / вдова,896,63,0.070312


Самая большая доля задолженности у тех кто **не женат / не замужем**. Но разница с теми кто в **гражданском браке** минимальна. И не очень велика с теми кто **женат / замужем**

### Вывод

Семейное положение не сильно влияет на способность вовремя платить по кредиту

### Выявление зависимости между уровнем дохода и возвратом кредита в срок <a id="income"></a>

In [41]:
data_pivot_income = data.pivot_table(index='income_group', columns='debt_group', values='debt', aggfunc='count')
data_pivot_income['income_part_debt'] = data_pivot_income['задолженность есть'] / (data_pivot_income['задолженность есть'] + data_pivot_income['задолженности нет'])
display(data_pivot_income.sort_values(by='income_part_debt', ascending = False))                


debt_group,задолженности нет,задолженность есть,income_part_debt
income_group,,,
доход от 100К до 200К,10894,1029,0.086304
доход от 50К до 100К,3760,331,0.080909
доход от 200К до 500К,4501,344,0.071001
доход > 500К,208,14,0.063063
доход < 50К,349,23,0.061828


Доля задолжности меньше всего у тех чей доход меньше 50К и большн 500К. А те у кого доход от 100 до 200К чаще имеют проблемы с выплатами, чем заемщики с доходом от 50 до 100К. 

Важнее не сколько человек зарабатывает, а как он тратит то что зарабатывает

### Вывод

Линейной зависимости между уровнем дохода и возвратом кредита в срок нет

### Выявление зависимости  как разные цели кредита влияют на его возврат в срок <a id="purpose"></a>

In [42]:
data_pivot_purpose = data.pivot_table(index='purpose_group', columns='debt_group', values='debt', aggfunc='count')
data_pivot_purpose['purpose_part_debt'] = data_pivot_purpose['задолженность есть'] / (data_pivot_purpose['задолженность есть'] + data_pivot_purpose['задолженности нет'])
display(data_pivot_purpose.sort_values(by='purpose_part_debt', ascending = False))

debt_group,задолженности нет,задолженность есть,purpose_part_debt
purpose_group,,,
автомобиль,3903,403,0.093590
образование,3643,370,0.092200
свадьба,2137,186,0.080069
недвижимость,10029,782,0.072334


Самая большая доля задолженностей приходится на автокредиты и образование. Машины быстро устаревают, могут сломаться, попасть в аварию, машину могут угнать. Плюс машина не всегда является предметом первой необходимости. И тогда у заемщика пропадает мотивация платить по кредиту.

Образование - нематериальный неосязаемый актив. Кредит на образование могут брать люди, которые до конца не опредлились чем хотят заниматься в жизни (например, студенты).

Меньше всего доля задолженностей  у кредитов на недвижимость. Недвижимость это постоянный материальный актив, который, как правило, со временем растет в цене. Это стимулирует заещика платить вовремя, особенно если это его единственное жилье.

### Вывод <a id="inference"></a>

Цель кредита потенциально влияет на возврат кредита в срок

##  4. Общий вывод

В ходе проведенного анализа выявлено, что наличие детей влияет на возврат кредита в срок. Заемщики, имеющие детей, чаще имеют проблемы с выплатой кредита в срок, чем бездетные заемщики. Просрочку кредита имели 9,2% заемщиков с детьми и 7,5% заемщиков без детей.

Также на способность вовремя платить по кредиту влияет и цель кредита. Чаще всего задерживают выплаты люди, взявшие кредит на автомобиль - таких оказалось 9,3%. Меньше всего задолженностей у заемщиков, взявших кредит на недвижимость - их 7,2%. 

Те кто не в браке или не оформили отношения официально чаще задерживают выплаты (≈ 10%) , чем те кто состоят в официальном браке или разведены (≈ 8%). 

Меньше всего должников среди заемщиков с доходом меньше 50К. Их только 6%.

Можно составить картину наиболее благонадежного заемщика: человек, состоящий в браке, без детей, берущий кредит на жилье, с доходом около 50К


В процессе обработки данных было выявлено некоторое кол-во пропущенных данных или некорректных данных. В определенных случаях это может привести к не точному анализу, поэтому заказчику можно было бы посоветовать:
- Добавить дополнительные данные по заемщику. Например, ФИО. Это бы облегчело определение грубых дубликатов.
- Для тех кто собирает данные сделать выпадающие списки там где имеется ограниченный выбор данных. Например, обазование, семейное положение, пол. Это так же уменьшит кол-во дубликатов.
- В данных, которые не могут быть отрицательными или равны 0, в системе поставить проверку на вводимые данные. Это данные о детях и возрасте.
- Так же следует обратить внимание на систему загрузки или выгрузки данных, чтобы определить где и из-за чего происходит сбой